# 「bert-as-serviceで文書ベクトル作成」の動作確認

In [1]:
import numpy as np
import os
import time
from bert_serving.server.helper import get_args_parser
from bert_serving.server.helper import get_shutdown_parser
from bert_serving.server import BertServer
from bert_serving.client import BertClient

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
DEBUG = False
TEXT_LIST = [
    'ロボットは人間に危害を加えてはならない。また、その危険を看過することによって、人間に危害を及ぼしてはならない。',
    'ロボットは人間にあたえられた命令に服従しなければならない。ただし、あたえられた命令が、第一条に反する場合は、この限りでない。',
    'ロボットは、前掲第一条および第二条に反するおそれのないかぎり、自己をまもらなければならない。',
    'すべての人間は、生れながらにして自由であり、かつ、尊厳と権利とについて平等である。',
    '人間は、理性と良心とを授けられており、互いに同胞の精神をもって行動しなければならない。',
    '「富士〜」で有名な動物園は？',
    '富士サファリパークです。',
    '東武動物公園です。',
    '軽井沢おもちゃ王国です。',
    'ロボットは機械です。',
]

## サーバ起動

In [3]:
os.environ['ZEROMQ_SOCK_TMP_DIR'] = '/app/tmp'
args = get_args_parser().parse_args([
    '-model_dir', '/app/model',
    '-ckpt_name', 'model.ckpt-1400000',
    '-config_name', 'bert-wiki-ja_config.json',
    '-graph_tmp_dir', '/app/tmp',
    '-port', '5555',
    '-port_out', '5556',
    '-max_seq_len', 'NONE',
    '-num_worker', '1',
    '-cpu',
    '-show_tokens_to_client',
])
server = BertServer(args)
server.start()

print('wait until server is ready...')
time.sleep(20)

I:VENTILATOR:[__i:__i: 67]:freeze, optimize and export graph, could take a while...
I:GRAPHOPT:[gra:opt: 52]:model config: /app/model/bert-wiki-ja_config.json
I:GRAPHOPT:[gra:opt: 55]:checkpoint: /app/model/model.ckpt-1400000
I:GRAPHOPT:[gra:opt: 59]:build graph...


I:GRAPHOPT:[gra:opt:128]:load parameters from checkpoint...
I:OPTIMIZE_GRAPH:[gra:opt:130]:visualize model with tensorboard
I:GRAPHOPT:[gra:opt:138]:optimize...
I:GRAPHOPT:[gra:opt:146]:freeze...
I:GRAPHOPT:[gra:opt:151]:write graph to a tmp file: /app/tmp/tmpat4nk8ji
I:VENTILATOR:[__i:__i: 75]:optimized graph is stored at: /app/tmp/tmpat4nk8ji
I:VENTILATOR:[__i:_ru:129]:bind all sockets
I:VENTILATOR:[__i:_ru:133]:open 8 ventilator-worker sockets
I:VENTILATOR:[__i:_ru:136]:start the sink


wait until server is ready...


I:VENTILATOR:[__i:_ge:222]:get devices
I:VENTILATOR:[__i:_ge:255]:device map: 
		worker  0 -> cpu
I:SINK:[__i:_ru:306]:ready
I:WORKER-0:[__i:_ru:533]:use device cpu, load graph from /app/tmp/tmpat4nk8ji


Loaded a trained SentencePiece model.


I:WORKER-0:[__i:gen:567]:ready and listening!
I:VENTILATOR:[__i:_ru:164]:all set, ready to serve request!
I:SINK:[__i:_ru:348]:send config	client b'199b2030-0372-4a4e-93c1-9ce37e50d7cb'
I:SINK:[__i:_ru:342]:job register	size: 1	job id: b'199b2030-0372-4a4e-93c1-9ce37e50d7cb#2'
I:WORKER-0:[__i:gen:576]:new job	socket: 0	size: 1	client: b'199b2030-0372-4a4e-93c1-9ce37e50d7cb#2'
W:WORKER-0:[ext:con: 64]:"max_seq_length" is undefined, and bert config json defines "max_position_embeddings"=512. hence set "max_seq_length"=30 according to the current batch.
I:SINK:[__i:_ru:334]:collect b'TOKENS' b'199b2030-0372-4a4e-93c1-9ce37e50d7cb#2' (E:0/T:1/A:1)
I:ESTIMATOR:[__i:_ru:545]:input_ids     
<class 'numpy.ndarray'>
(1, 30)
[[    4     9  4018    11   858    17 14411  5596  8660 17172     8   240
      7    65  3657    18 17527  2377 10545     7   858    17 14411  5596
     18 20016    32 17172     8     5]]
I:ESTIMATOR:[__i:_ru:546]:input_mask    
<class 'numpy.ndarray'>
(1, 30)
[[1 1 1 1 1 1 

## 文書ベクトル取得

In [4]:
bc = BertClient()
doc_vecs = []
for text in TEXT_LIST:
    result = bc.encode([text], show_tokens=True)
    embeddings = result[0][0]
    tokens = result[1][0]
    doc_vecs.append(embeddings)

    if DEBUG:
        print('########################################')
        print('text        :', text)
        print('tokens      :', tokens)
        print('len(tokens) :', len(tokens))
        print('type        :', type(embeddings))
        print('shape       :', embeddings.shape)
        print('norm        :', np.linalg.norm(embeddings))
        print('embeddings  :', '[', embeddings[0], embeddings[1], '...', embeddings[-2], embeddings[-1], ']')
        print()
bc.close()

I:VENTILATOR:[__i:_ru:180]:new config request	req id: 1	client: b'199b2030-0372-4a4e-93c1-9ce37e50d7cb'
/usr/local/lib/python3.6/dist-packages/bert_serving_client-1.9.5-py3.6.egg/bert_serving/client/__init__.py:290: UserWarning: server does not put a restriction on "max_seq_len", it will determine "max_seq_len" dynamically according to the sequences in the batch. you can restrict the sequence length on the client side for better efficiency
  warnings.warn('server does not put a restriction on "max_seq_len", '
I:VENTILATOR:[__i:_ru:196]:new encode request	req id: 2	size: 1	client: b'199b2030-0372-4a4e-93c1-9ce37e50d7cb'
I:VENTILATOR:[__i:_ru:196]:new encode request	req id: 3	size: 1	client: b'199b2030-0372-4a4e-93c1-9ce37e50d7cb'
I:VENTILATOR:[__i:_ru:196]:new encode request	req id: 4	size: 1	client: b'199b2030-0372-4a4e-93c1-9ce37e50d7cb'
I:VENTILATOR:[__i:_ru:196]:new encode request	req id: 5	size: 1	client: b'199b2030-0372-4a4e-93c1-9ce37e50d7cb'
I:VENTILATOR:[__i:_ru:196]:new encode

## サーバ停止

In [5]:
server.close()

I:VENTILATOR:[__i:clo: 89]:shutting down...
I:SINK:[__i:clo:274]:shutting down...
I:SINK:[__i:clo:279]:terminated!
I:WORKER-0:[__i:clo:480]:shutting down...
I:WORKER-0:[__i:clo:485]:terminated!
I:VENTILATOR:[__i:_ru:219]:terminated!


In [6]:
query_vec = doc_vecs[0]
score = np.sum(query_vec * doc_vecs, axis=1) / np.linalg.norm(doc_vecs, axis=1) / np.linalg.norm(query_vec)
topk_idx = np.argsort(score)[::-1]
for idx in topk_idx:
    print('{:6.4f} : {}'.format(score[idx], TEXT_LIST[idx]))

1.0000 : ロボットは人間に危害を加えてはならない。また、その危険を看過することによって、人間に危害を及ぼしてはならない。
0.9205 : ロボットは人間にあたえられた命令に服従しなければならない。ただし、あたえられた命令が、第一条に反する場合は、この限りでない。
0.8995 : ロボットは、前掲第一条および第二条に反するおそれのないかぎり、自己をまもらなければならない。
0.8914 : 人間は、理性と良心とを授けられており、互いに同胞の精神をもって行動しなければならない。
0.8719 : すべての人間は、生れながらにして自由であり、かつ、尊厳と権利とについて平等である。
0.8482 : ロボットは機械です。
0.8270 : 「富士〜」で有名な動物園は？
0.8017 : 東武動物公園です。
0.7916 : 軽井沢おもちゃ王国です。
0.7887 : 富士サファリパークです。
